<a href="https://colab.research.google.com/github/potryasas/LTSC/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%A0%D1%83%D1%81%D0%B8%D1%84%D0%B8%D1%86%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D1%8B%D0%B9_%D1%82%D1%80%D0%B5%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%BE%D1%87%D0%BD%D1%8B%D0%B9_Colab_2_0_(t_me_lnhly)%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Русифицированный Colab от [lunnaholy](https://t.me/lnhly)**

Колаб для генерации вокала: [тык](https://colab.research.google.com/drive/1o1gtZ5djnuYjFhAPx06CZyESkW9Ofk9P)

# Первоначальная настройка

In [ ]:
#@title Установка зависимостей
!git clone https://github.com/effusiveperiscope/so-vits-svc -b eff-4.0

%cd /content/so-vits-svc
!pip install pyworld praat-parselmouth
!python -m pip install --upgrade pip
!pip install fairseq==0.12.2 librosa==0.8.1 numpy==1.23.5
!wget -P hubert/ https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/checkpoint_best_legacy_500.pt

In [ ]:
#@title Подсоединение Google Диска
from google.colab import drive
drive.mount('/content/drive')

# Обработка датасета

Можете пропустить этот раздел, если вы уже имеете модель и хотите продолжить тренировку.

In [ ]:
#@title Загрузка датасета
#@markdown Название архива
DATASETNAME = "dataset"  #@param {type:"string"}
#@markdown Путь к папке (не нужно менять, если грузите по гайду)
ZIP_PATH = "/content/drive/MyDrive/dataset/"  #@param {type:"string"}
ZIP_NAME = ZIP_PATH + DATASETNAME

!unzip -d /content/so-vits-svc/dataset_raw {ZIP_NAME}.zip

In [ ]:
#@title Обработка датасета
!python resample.py
!python preprocess_flist_config.py
!python preprocess_hubert_f0.py

In [ ]:
#@title Сохранение обработанного датасета
#Compress dataset folder
!zip -r dataset.zip /content/so-vits-svc/dataset
#@markdown Название датасета
dataset_name_drive = "dataset"  #@param {type:"string"}
DATASET_PATH_DRIVE = "/content/drive/MyDrive/dataset/" + dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp /content/so-vits-svc/dataset.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.txt "{DATASET_PATH_DRIVE}"
!cp filelists/val.txt "{DATASET_PATH_DRIVE}"

# Тренировка

In [ ]:
#@title Загрузить обработанный датасет
#@markdown Если вы продолжаете тренировку, то вы можете просто загрузить уже обработанный датасет с диска.
back_up_name = "dataset"  #@param {type:"string"}
BACK_UP_DATASET_PATH = "/content/drive/MyDrive/dataset/" + back_up_name
!unzip {BACK_UP_DATASET_PATH}/dataset.zip -d /
!cp {BACK_UP_DATASET_PATH}/config.json /content/so-vits-svc/configs/config.json
!cp {BACK_UP_DATASET_PATH}/val.txt filelists/val.txt
!cp {BACK_UP_DATASET_PATH}/train.txt filelists/train.txt

In [ ]:
#@title Предварительные настройки
Clone = "44k"
%cd /content/so-vits-svc

#@markdown **Сохраните файлы обученной модели в Google Drive вместо файловой системы среды выполнения colab. Вам также необходимо проверить и выполнить эту опцию при возобновлении обучения. Если у вас мало места на диске, вы можете снять этот флажок (в этом случае вы должны использовать меню файл, чтобы вручную загрузить контрольные точки модели в so-vits-svc/logs/ и скопировать последнюю контрольную точку обратно в so-vits-svc/logs/ для возобновления обучения).**
Save_to_drive = True #@param {type:"boolean"}
if Save_to_drive:
  !rm -rf /content/so-vits-svc/logs/"{Clone}"
  !mkdir -p /content/drive/MyDrive/"{Clone}"
  !ln -s /content/drive/MyDrive/"{Clone}" /content/so-vits-svc/logs/"{Clone}"

#@markdown **Загрузить предварительно обученую модель. НЕ ВЫПОЛНЯТЬ ПРИ ПРОДОЛЖЕНИИ ТРЕНИРОВКИ!!!**
pre_pth = True #@param {type:"boolean"}
if pre_pth:
  !wget -O logs/"{Clone}"/G_0.pth -P logs/"{Clone}"/ https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/G_0.pth
  !wget -O logs/"{Clone}"/D_0.pth -P logs/"{Clone}"/ https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/D_0.pth


In [ ]:
#@title Запустить тренировку
#@markdown **Запустить тренировку**
Clone = "44k"

#@markdown **Включить Tensorboard для визуализации прогресса**
tensorboard_on = True #@param {type:"boolean"}
if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/"{Clone}"

!python train.py -c configs/config.json -m "{Clone}"
